# ニューラルネットワークの訓練: 重みの更新回数と出力値

参考コード：
https://github.com/pytorch/examples/blob/master/snli/train.py


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import time

import pandas as pd
import matplotlib.pyplot as plt

import torch as t
import torch.optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv

各種関数を定義

In [ ]:
device = t.device("cuda:0" if t.cuda.is_available() else "cpu")
device

In [ ]:
class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()
        self.l1 = nn.Linear(784, 1000) 
        self.l2 = nn.Linear(1000, 1000)
        self.l3 = nn.Linear(1000, 10)

    def forward(self, x):
        h = x.view(-1, 28*28) # (N, 1, 28, 28) -> (N, 784)
        # １層目
        h = F.relu(self.l1(h))
        # ２層目
        h = F.relu(self.l2(h))
        # 出力層
        return self.l3(h)

In [ ]:
def dev_env(tensor):
    return "cuda" if tensor.is_cuda else "cpu"

今回は`running_loss`をコードの可読性のためリストとして定義した。  
だが、各lossをメモリに展開しておく理由も特にないので、普通のfloat変数として定義して毎回加算してしまって問題ない。  

In [ ]:
def train(model, trainset, criterion, lr, batchsize, iterations):
    """
    指定されたモデルを指定回数分だけ訓練させる関数。
    """
    model.train()
    train_loader = t.utils.data.DataLoader(trainset, batch_size=batchsize)
    optimizer = t.optim.Adam(model.parameters(), lr=lr)
    running_loss = []
    correct = 0
    axis = 1
    if iterations is None:
        iterations = len(train_loader.dataset.data)
    criterion = criterion.to(device)
    model = model.to(device)
    for i, (inputs, labels) in enumerate(train_loader):
        # Move device
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        # Forward + Backward
        p = model(inputs)
        loss = criterion(p, labels)
        loss.backward()
        optimizer.step()
        # Calc statistics
        running_loss.append(loss.item())
        predicted_labels = p.max(axis)[1]
        correct += (predicted_labels == labels).sum().item() # 一致数の算出
        # Skip code
        if iterations < (i +1):
            break
    train_loss = np.array(running_loss).mean()
    train_acc = float(correct) / trainset.data.shape[0]
    return train_loss, train_acc

In [ ]:
def valid(model, testset, criterion, batchsize):
    """
    指定されたモデルでデータを検証する関数
    """
    model.eval()
    model = model.to(device)
    criterion = criterion.to(device)
    correct = 0
    total = 0
    axis = 1
    running_loss = []
    test_loader = t.utils.data.DataLoader(testset, batchsize)
    with t.no_grad(): # Disable to calc gradient
        for i, (inputs, labels) in enumerate(test_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            # Forward + Backward
            p = model(inputs) # p is one-hot vector
            loss = criterion(p, labels)
            # Calc statistics
            running_loss.append(loss.item())
            predicted_labels = p.max(axis)[1]
            correct += (predicted_labels == labels).sum().item()
    val_loss = np.array(running_loss).mean()
    val_acc = float(correct) / testset.data.shape[0]
    return val_loss, val_acc

## ポイント1

```python
correct =  (predicted == labels).sum().item()
```
`(predicted_labels == labels)`でラベルが一致したかどうかの配列を生成している。  
一致している場合は`True`が格納されて、そうでない場合は`False`が格納される。  
その`True`の数を総和して取得している。

> itemメソッドは`Tensor`型に含まれている単一数字を取得できる。

## ポイント2

```python
total += labels.size(0)
```

`labels`は`torch.tensor`型なので`labels.size()`を実行しても`Tensor([n])`が取得できてしまって数値演算できない。  
よって、0次元目の値を取り出すために`tensor.size(0)`でアクセスしている。  
基本的に`labels.size(0)`は`DataLoader`が返す`Enumerarte`のサイズなので**基本的には**バッチサイズと一致する。  
ただし、batchsizeによっては端数が出る可能性があるので、`tensor.size(0) * i`として`O(1)`計算するのは難しい。

> 今回はデータセット数10,000に対してbatchsize=100でアクセスしているので端数が出ることなくアクセスできている。  
> しかし、batchsize=128などでアクセスした場合は余りが発生するのでそうはならない。最後の`labels.size(0)`は16となる。


In [ ]:
csv_path = '/content/drive/My Drive/ColabNotebooks/PyTorchBeginner/loss.csv'

In [ ]:
def train_and_eval(model, trainset, testset, lr=0.001, epochs=1, batchsize=100, iterations=None):
    results = []
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        start = time.time()
        train_loss, train_acc = train(model, trainset, criterion, lr, batchsize, iterations)
        valid_loss, valid_acc = valid(model, trainset, criterion, batchsize)
        end = time.time()
        results.append([epoch, train_loss, train_acc, valid_loss, valid_acc, end - start])
        print('epoch %d, train_loss: %.4f train_acc: %.4f val_loss: %.4f val_acc: %.4f' % (epoch, train_loss, train_acc, valid_loss, valid_acc))
    out = pd.DataFrame(results, columns=["epoch", "train_loss", "train_accuracy", "valid_loss", "valid_accuracy", "elapsed_time"])
    out.to_csv(csv_path, index=False)
    return out


データをロードして準備しましょう。

In [ ]:
 preprocess = tv.transforms.Compose([
                                  tv.transforms.ToTensor()
                                  ])

In [ ]:
trainset = tv.datasets.MNIST('~/tmp/mnist', 
                                           train=True,
                                           download=True,
                                           transform=preprocess)

In [ ]:
testset = tv.datasets.MNIST('~/tmp/mnist',
                                          train=False,
                                          download=True,
                                          transform=preprocess)

## トレーニングを繰り返す

In [ ]:
batchsize=100

In [ ]:
result = train_and_eval(model=MLP(), trainset=trainset, testset=testset, epochs=20)

In [ ]:
result.head()

## 結果のグラフ化

各損失関数の値(loss)を表示

In [ ]:
result[['train_loss', 'valid_loss']].plot()

accuracyの表示

In [ ]:
result[['train_accuracy', 'valid_accuracy']].plot()

訓練データに対する正答率、検証データに対する正答率ともにキレイなグラフで増加しています。  
特に過学習が進んでいる様子もないため、精度の良いモデルができたと言えるでしょう。